In [ ]:
import torch

from mlmodule.contrib.mtcnn import MTCNNDetector
from mlmodule.contrib.arcface import ArcFaceFeatures
from mlmodule.torch.data.images import ImageDataset
from mlmodule.utils import list_files_in_dir


In [ ]:
# load models
device = torch.device('cuda:1')
arcface = ArcFaceFeatures(device=device).load()
mtcnn = MTCNNDetector(device=device, image_size=720, min_face_size=20).load()

In [ ]:
# run face detection first
base_path = "../tests/fixtures/berset"
file_names = list_files_in_dir(base_path, allowed_extensions=('jpg',))
dataset = ImageDataset(file_names)
# Detect faces first
file_names, outputs = mtcnn.bulk_inference(dataset)

In [ ]:
from mlmodule.contrib.arcface import ArcFaceAlignment
from mlmodule.torch.data.faces import FaceDataset

# load dataset with face descriptor
face_dataset = FaceDataset(file_names, outputs, crop_fn=ArcFaceAlignment())

In [ ]:
from PIL import Image
from matplotlib import pyplot as plt
# display cropped faces
for face in face_dataset:
    plt.figure()
    plt.imshow(Image.fromarray(face[1]))
    plt.show()

In [ ]:
# get data loader
loader = arcface.get_data_loader(face_dataset, batch_size=3)
# get batch
indices, batch = next(iter(loader))

In [ ]:
# get features
arcface.eval()
with torch.no_grad():
    outputs = arcface(batch.to(device))